# QUIZ - 2024-08

In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from django.db.models import *
from django.db.models.functions import *
from django.db.models.lookups import *

from service.models import *

## PART 1 - CRUD Operations
1.1 สร้างนัดหมาย (`Appointment`) ของลูกค้า "Customer 1" ซึ่งต้องการจองบริการ "Haircut" ของ "Provider 2"

โดยสร้างนัดหมายในวันที่ 2024-10-01 เวลา 11:00 น. 

(0.5 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [41]:
from datetime import datetime

cus1 = Customer.objects.get(name="Customer 1")
provider2 = Service.objects.get(name="Haircut", service_provider__name="Provider 2")


appoint = Appointment.objects.create(customer=cus1, service= provider2, appointment_date='2024-10-01', appointment_time = '11:00:00')



In [42]:
# Check result
res = Appointment.objects.last()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-10-01, Appoint Time: 11:00:00
Service Name: Haircut, Provider Name: Provider 2, Customer Name: Customer 1


1.2 ทำการแก้ไขนัดหมายที่เพิ่งสร้างใน 1.1 โดยเปลี่ยนวันที่จากวันที่ 2024-10-01 เป็น 2024-10-10 เวลาเดิม

(0.25 คะแนน)

In [43]:
appoint.appointment_date = '2024-10-10'
appoint.save()

In [44]:
# Check result
res = Appointment.objects.last()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-10-10, Appoint Time: 11:00:00
Service Name: Haircut, Provider Name: Provider 2, Customer Name: Customer 1


1.3 ทำการแก้ไขนัดหมาย ณ วันที่ 2024-08-14 ของ "Customer 4" โดยเปลี่ยนจากเดิมที่นัดมาให้บริการ "Manicure" ของ "Provider 3" เป็น "Massage" ของ "Provider 4" แทน

(0.25 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [45]:
cus = Customer.objects.get(name="Customer 4")
service_old = Service.objects.get(name="Manicure", service_provider__name="Provider 3")
service_new = Service.objects.get(name="Massage", service_provider__name="Provider 4")

app4 = Appointment.objects.get(customer = cus, service=service_old)
app4.service = service_new
app4.save()

In [47]:
# Check result
from datetime import date

res = Appointment.objects.get(customer__name="Customer 4", appointment_date=date(2024,8,14))
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-08-14, Appoint Time: 10:00:00
Service Name: Massage, Provider Name: Provider 4, Customer Name: Customer 4


1.4 ทำตามขั้นตอนดังนี้

1. สร้างหมวดหมู่บริการ (`ServiceCategory`) ใหม่ชื่อ "Facial & Body Care" และมีรายละเอียดว่า "Services related to face & body care" 
2. สร้าง บริการ (`Service`) ใหม่ดังนี้ 

    - Name: "Face Massage"
    - Description: "Special face massage!"
    - Price: 150.00
    - โดยบริการนี้ให้บริการโดย "Provider 1" และ "Provider 2" (Hint: ดังนั้นต้องสร้าง 2 รายการ)

3. เพิ่มบริการใหม่ทั้ง 2 รายการเข้าไปในหมวดหมู่ "Facial & Body Care"
4. ทำการย้ายบริการ (`Service`) ในหมวดหมู่บริการ (`ServiceCategory`) "Body Care" ไปที่ หมวดหมู่ "Facial & Body Care" ที่สร้างขึ้นมาใหม่
5. ทำการลบหมวดหมู่บริการ "Body Care"

(1 คะแนน)

In [49]:
provider1 = ServiceProvider.objects.get(name = 'Provider 1')
provider2 = ServiceProvider.objects.get(name = 'Provider 2')

category_body_cate = ServiceCategory.objects.create(name="Facial & Body Care", description="Services related to face & body care")
new_service = Service.objects.create(service_provider = provider1, name="Face Massage", description="Special face massage!", price = 150.00)
new_service = Service.objects.create(service_provider = provider2, name="Face Massage", description="Special face massage!", price = 150.00)

category_body_cate = ServiceCategory.objects.get(name="Facial & Body Care")
new_service1 = Service.objects.get(service_provider = provider1, name = "Face Massage")
new_service2 = Service.objects.get(service_provider = provider2, name = "Face Massage")
category_body_cate.services.add(new_service1, new_service2)


cate_body_care = Service.objects.filter(servicecategory__name = "Body Care")

for i in cate_body_care:
    category_body_cate.services.add(i)


category_bodycare_cate = ServiceCategory.objects.get(name="Body Care")
category_bodycare_cate.services.clear()
category_bodycare_cate.delete()

(1, {'service.ServiceCategory': 1})

In [52]:
# Check result
sc1 = ServiceCategory.objects.get(name="Facial & Body Care")
sc1.services.count()

7

## PART 2 - Making Queries

สำหรับ PART 2 ให้ทำการ reset DB และ import ข้อมูลใน `service.sql` เข้าไปใหม่

2.1 ให้หาจำนวนนัดหมาย ที่ถูกสร้าง (`Appointment.created_at`) ในช่วงวันที่ 2024-08-01 ถึง 2024-08-02 

(0.5 คะแนน)

In [7]:
Appointment.objects.filter(created_at__gte = "2024-08-01", created_at__lte = "2024-08-02").count()

2

2.2 ให้หาว่ามีบริการใด ของผู้ให้บริการไหนบ้าง ที่มีการนัดหมายท่ีมีการนัดหมาย (`Appointment.appoint_date`) ในช่วงวันที่ 2024-08-10 ถึง 2024-08-15

(0.5 คะแนน)

**Expected Output**

```
Name: Haircut, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Massage, Provider: Provider 2
Name: Massage, Provider: Provider 3
Name: Massage, Provider: Provider 4
```

In [15]:
service = Service.objects.filter(appointment__appointment_date__gte = "2024-08-10", appointment__appointment_date__lte = "2024-08-15")

for i in service:
    print("Name: "+i.name+", Provider:", i.service_provider.name)
    



Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2
Name: Manicure, Provider: Provider 2
Name: Manicure, Provider: Provider 2
Name: Massage, Provider: Provider 2
Name: Manicure, Provider: Provider 3
Name: Massage, Provider: Provider 3


2.3 ให้แสดงผลข้อมูลบริการ (`Service`) โดยเรียงลำดับตามราคา (`Service.price`) จากน้อยไปมาก ดังในตัวอย่าง

```python
[
    {
        "price": 100,
        "full_name": "Massage (Provider 1)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 2)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 3)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 4)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 4)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 1)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 3)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 2)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 4)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 2)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 3)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 1)"
    }
]
```

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()` เท่านั้น การ loop และสร้าง dict เองจะไม่ได้คะแนน*

Hint: อยาก print dictionary สวยๆ ใช้ json.dumps

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

In [20]:
import json

services = Service.objects.annotate(full_name=Concat(('name'), Value('('),('service_provider__name'),Value(')'))).all().values('price', 'full_name').order_by('-price')

for i in services:
    print(json.dumps(i, indent=4, sort_keys=False))

{
    "price": 100,
    "full_name": "Massage(Provider 4)"
}
{
    "price": 100,
    "full_name": "Massage(Provider 2)"
}
{
    "price": 100,
    "full_name": "Massage(Provider 1)"
}
{
    "price": 100,
    "full_name": "Massage(Provider 3)"
}
{
    "price": 30,
    "full_name": "Manicure(Provider 3)"
}
{
    "price": 30,
    "full_name": "Manicure(Provider 1)"
}
{
    "price": 30,
    "full_name": "Manicure(Provider 4)"
}
{
    "price": 30,
    "full_name": "Manicure(Provider 2)"
}
{
    "price": 20,
    "full_name": "Haircut(Provider 3)"
}
{
    "price": 20,
    "full_name": "Haircut(Provider 2)"
}
{
    "price": 20,
    "full_name": "Haircut(Provider 1)"
}
{
    "price": 20,
    "full_name": "Haircut(Provider 4)"
}


2.4 ให้หาจำนวนนัดหมายของลูกค้าแต่ละคน

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()`*

**Expected Output**

```python
Name: Customer 4, Total Appoinment: 3
Name: Customer 2, Total Appoinment: 4
Name: Customer 3, Total Appoinment: 3
Name: Customer 1, Total Appoinment: 5
```

In [30]:
cus = Customer.objects.values('name').annotate(Total_Appoinment = Count('appointment__id'))

for i in cus:
    print("Name: "+i['name']+ ", Total Appoinment: "+str(i['Total_Appoinment']))

Name: Customer 2, Total Appoinment: 4
Name: Customer 4, Total Appoinment: 3
Name: Customer 3, Total Appoinment: 3
Name: Customer 1, Total Appoinment: 5


2.5 ให้หาว่าผู้ให้บริการ (`ServiceProvider`) แต่ละคนนั้นได้เงินโดยรวมเท่าไหร่

Hint: ดูจากนั้ดหมายว่ามีการนัดบริการ (`Appointment.service`) ของแต่ละผู้ให้บริการรวมเป็นยอดเงินเท่าไหร่

*หมายเหตุ: ไม่จำเป็นต้องใช้ `annotate()` สามารถ loop ได้เต็มที่เลยครับ*

**Expected Output**

```python
Name: Provider 4, Total Price: 20
Name: Provider 2, Total Price: 210
Name: Provider 3, Total Price: 130
Name: Provider 1, Total Price: 410
```

In [37]:
provider = Appointment.objects.values('service__service_provider__name').annotate(total = Sum('service__price'))

for i in provider:
    print("Name: "+ i['service__service_provider__name']+", Total Price: "+str(i['total']))

Name: Provider 1, Total Price: 410
Name: Provider 4, Total Price: 20
Name: Provider 2, Total Price: 210
Name: Provider 3, Total Price: 130


2.6 ให้ดึงข้อมูลหมวดหมู่บริการโดยแปลงชื่อหมวดหมู่บริการ (`ServiceCategory.name`) เป็นตัวใหญ่ (uppercase) ทั้งหมด

(0.5 คะแนน)

**Expected Output**

```python
Name: Hair Care, Upper Name: HAIR CARE
Name: Nail Care, Upper Name: NAIL CARE
Name: Body Care, Upper Name: BODY CARE
```

In [40]:
service_cate = ServiceCategory.objects.annotate(upper = Upper('name')).all()

for i in service_cate:
    print("Name: "+i.name+", Upper Name: "+i.upper)

Name: Hair Care, Upper Name: HAIR CARE
Name: Nail Care, Upper Name: NAIL CARE
Name: Body Care, Upper Name: BODY CARE
